In [1]:
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv', null_values=["NA"])
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values=["NA"])
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values=["NA"])
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values=["NA"], infer_schema_length=10000)

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))
print("Weather:", weather.columns)
print("Flights:", flights.columns)
print("Planes:", planes.columns)
print("Airlines:", airlines.columns)
print("Airports:", airports.columns)

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘
Weather: ['origin', 'year', 'month', 'day', 'hour', 'temp', 'dewp', 'humid', 'wind_dir', 'wind_speed', 'wind_gust', 'precip', 'pressure', 'visib', 'time_hour']
Flights: ['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight', 'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute', 'time_hour']
Planes: ['tailnum', 'year', 'type', 'manufacturer', 'model', 'engines', 'seats', 'speed', 'engine']
Airlines: ['carrier', 'name']
Airports: ['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst', 'tzone']


/tmp/ipython-input-1938067338.py:14: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


In [2]:
#Exercise 1.1
result = ctx.execute("""
select distinct carrier
from airlines
""")

result

carrier
str
"""9E"""
"""AA"""
"""AS"""
"""B6"""
"""DL"""
…
"""UA"""
"""US"""
"""VX"""


In [3]:
#Exercise 1.2
result = ctx.execute("""
select dest, count (*) as count
from flights
group by dest
order by count desc
limit 10
""")

result

dest,count
str,u32
"""ORD""",17283
"""ATL""",17215
"""LAX""",16174
"""BOS""",15508
"""MCO""",14082
"""CLT""",14064
"""SFO""",13331
"""FLL""",12055
"""MIA""",11728


In [4]:
#Exercise 1.3
result = ctx.execute("""
select *
from flights
where dep_delay > 120
""")

result

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,str,str,i64,i64,i64,i64,"datetime[μs, UTC]"
2013,1,1,848,1835,853,1001,1950,851,"""MQ""",3944,"""N942MQ""","""JFK""","""BWI""",41,184,18,35,2013-01-01 23:00:00 UTC
2013,1,1,957,733,144,1056,853,123,"""UA""",856,"""N534UA""","""EWR""","""BOS""",37,200,7,33,2013-01-01 12:00:00 UTC
2013,1,1,1114,900,134,1447,1222,145,"""UA""",1086,"""N76502""","""LGA""","""IAH""",248,1416,9,0,2013-01-01 14:00:00 UTC
2013,1,1,1540,1338,122,2020,1825,115,"""B6""",705,"""N570JB""","""JFK""","""SJU""",193,1598,13,38,2013-01-01 18:00:00 UTC
2013,1,1,1815,1325,290,2120,1542,338,"""EV""",4417,"""N17185""","""EWR""","""OMA""",213,1134,13,25,2013-01-01 18:00:00 UTC
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2013,9,30,1823,1545,158,1934,1733,121,"""9E""",3459,"""N916XJ""","""JFK""","""BNA""",95,765,15,45,2013-09-30 19:00:00 UTC
2013,9,30,1951,1649,182,2157,1903,174,"""EV""",4294,"""N13988""","""EWR""","""SAV""",95,708,16,49,2013-09-30 20:00:00 UTC
2013,9,30,2053,1815,158,2310,2054,136,"""EV""",5292,"""N600QX""","""EWR""","""ATL""",91,746,18,15,2013-09-30 22:00:00 UTC


In [5]:
#Exercise 2.1
result = ctx.execute("""
SELECT origin, AVG(dep_delay) AS avg_dep_delay
FROM flights
GROUP BY origin
ORDER BY avg_dep_delay DESC;
""")

result

origin,avg_dep_delay
str,f64
"""EWR""",15.107954
"""JFK""",12.112159
"""LGA""",10.346876


In [6]:
#Exercise 2.2
result = ctx.execute("""
    SELECT *
    FROM flights
    LIMIT 5
""")

result = ctx.execute("""
select month, count(*) as count
from flights
group by month
order by count desc
""")

result

month,count
i64,u32
7,29425
8,29327
10,28889
3,28834
5,28796
…,…
12,28135
9,27574
11,27268


In [7]:
#Exercise 2.3
result = ctx.execute("""
SELECT carrier, ROUND(
        100.0 * SUM(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) / COUNT(*),
        2
    ) AS on_time_rate
FROM flights
GROUP BY carrier
ORDER BY on_time_rate DESC;
""")

result

carrier,on_time_rate
str,f64
"""HA""",92.98
"""AS""",86.55
"""US""",84.99
"""DL""",83.07
"""AA""",82.44
…,…
"""FL""",71.69
"""F9""",71.53
"""9E""",70.12


In [ ]:
#Exercise 3.1
result = ctx.execute("""
SELECT
    f.carrier,
    a.name AS airline_name,
    f.flight AS flight_number,
    f.origin,
    f.dest
FROM flights f
JOIN airlines a
    ON f.carrier = a.carrier
LIMIT 20;
""")

result

carrier,airline_name,flight_number,origin,dest
str,str,i64,str,str
"""UA""","""United Air Lines Inc.""",1545,"""EWR""","""IAH"""
"""UA""","""United Air Lines Inc.""",1714,"""LGA""","""IAH"""
"""AA""","""American Airlines Inc.""",1141,"""JFK""","""MIA"""
"""B6""","""JetBlue Airways""",725,"""JFK""","""BQN"""
"""DL""","""Delta Air Lines Inc.""",461,"""LGA""","""ATL"""
…,…,…,…,…
"""B6""","""JetBlue Airways""",1806,"""JFK""","""BOS"""
"""UA""","""United Air Lines Inc.""",1187,"""EWR""","""LAS"""
"""B6""","""JetBlue Airways""",371,"""LGA""","""FLL"""


In [8]:
#Exercise 3.2
result = ctx.execute("""
SELECT
    f.carrier,
    ROUND(AVG(2013 - p.year), 1) AS avg_plane_age
FROM flights f
JOIN planes p
    ON f.tailnum = p.tailnum
WHERE p.year IS NOT NULL
GROUP BY f.carrier
ORDER BY avg_plane_age DESC;
""")

result

carrier,avg_plane_age
str,f64
"""MQ""",35.3
"""AA""",25.9
"""DL""",16.4
"""UA""",13.2
"""FL""",11.4
…,…
"""B6""",6.7
"""F9""",4.9
"""VX""",4.5


In [9]:
#Exercise 3.3
result = ctx.execute("""
    SELECT *
    FROM weather
    LIMIT 5
""")
result

result = ctx.execute("""
SELECT
    f.flight,
    f.carrier,
    f.origin,
    f.dest,
    f.dep_delay,
    w.wind_speed,
    w.precip
FROM flights f
JOIN weather w
    ON f.origin = w.origin
   AND f.year = w.year
   AND f.month = w.month
   AND f.day = w.day
   AND f.hour = w.hour
WHERE f.dep_delay > 30
  AND (w.wind_speed > 20 OR w.precip > 0.1);""")

result

flight,carrier,origin,dest,dep_delay,wind_speed,precip
i64,str,str,str,i64,f64,f64
21,"""B6""","""JFK""","""TPA""",47,21.86482,0.0
199,"""B6""","""JFK""","""LAS""",116,21.86482,0.0
4090,"""EV""","""EWR""","""JAX""",39,20.71404,0.0
4231,"""EV""","""EWR""","""IAD""",40,24.16638,0.0
1010,"""B6""","""JFK""","""BOS""",33,20.71404,0.0
…,…,…,…,…,…,…
2347,"""DL""","""LGA""","""ATL""",41,20.71404,0.0
5503,"""EV""","""LGA""","""PIT""",97,20.71404,0.0
135,"""B6""","""JFK""","""PHX""",42,21.86482,0.0


In [10]:
#Exercise 4.1
result = ctx.execute("""
SELECT
    p.manufacturer,
    p.model,
    COUNT(*) AS flight_count
FROM flights f
JOIN planes p
    ON f.tailnum = p.tailnum
WHERE p.manufacturer IS NOT NULL
  AND p.model IS NOT NULL
GROUP BY p.manufacturer, p.model
ORDER BY flight_count DESC
LIMIT 10
""")

result

manufacturer,model,flight_count
str,str,u32
"""AIRBUS""","""A320-232""",31278
"""EMBRAER""","""EMB-145LR""",28027
"""EMBRAER""","""ERJ 190-100 IGW""",23716
"""AIRBUS INDUSTRIE""","""A320-232""",14553
"""EMBRAER""","""EMB-145XR""",14051
"""BOEING""","""737-824""",13809
"""BOMBARDIER INC""","""CL-600-2D24""",11807
"""BOEING""","""737-7H4""",10389
"""BOEING""","""757-222""",9150


In [11]:
#Exercise 4.2
result = ctx.execute("""
WITH FlightDetails AS (
    SELECT
        r.origin,
        ao.name AS origin_airport,
        r.dest,
        ad.name AS dest_airport,
        r.dep_delay
    FROM flights r
    JOIN airports ao ON r.origin = ao.faa
    JOIN airports ad ON r.dest = ad.faa
)
SELECT
    origin,
    origin_airport,
    dest,
    dest_airport,
    COUNT(*) AS total_flights,
    ROUND(AVG(dep_delay), 2) AS avg_dep_delay,
    ROUND(
        100.0 * SUM(CASE WHEN dep_delay > 30 THEN 1 ELSE 0 END) / COUNT(*),
        2
    ) AS pct_delayed_over_30
FROM FlightDetails
GROUP BY origin, origin_airport, dest, dest_airport
ORDER BY total_flights DESC
LIMIT 10
""")

result

origin,origin_airport,dest,dest_airport,total_flights,avg_dep_delay,pct_delayed_over_30
str,str,str,str,u32,f64,f64
"""JFK""","""John F Kennedy Intl""","""LAX""","""Los Angeles Intl""",11262,8.52,9.83
"""LGA""","""La Guardia""","""ATL""","""Hartsfield Jackson Atlanta Int…",10263,11.45,12.25
"""LGA""","""La Guardia""","""ORD""","""Chicago Ohare Intl""",8857,10.74,13.35
"""JFK""","""John F Kennedy Intl""","""SFO""","""San Francisco Intl""",8204,11.95,12.12
"""LGA""","""La Guardia""","""CLT""","""Charlotte Douglas Intl""",6168,8.97,11.95
"""EWR""","""Newark Liberty Intl""","""ORD""","""Chicago Ohare Intl""",6100,14.64,16.0
"""JFK""","""John F Kennedy Intl""","""BOS""","""General Edward Lawrence Logan …",5898,11.69,13.77
"""LGA""","""La Guardia""","""MIA""","""Miami Intl""",5781,7.36,9.46
"""JFK""","""John F Kennedy Intl""","""MCO""","""Orlando Intl""",5464,10.6,12.72
